In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as  plt
%matplotlib inline

In [2]:
from wordcloud import WordCloud

In [3]:
import nltk
from nltk.corpus import stopwords

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_csv(r"D:\ML-Ops\dvc-pipeline\experiments\spam.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'experiments\\spam.csv'

In [ ]:
df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace= True)
df.head()

In [ ]:
df.rename(columns={'v1':'target','v2':'text'}, inplace=True)
df.head()

Data Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['target'] = encoder.fit_transform(df['target'])
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
len(df)

In [ ]:
df = df.drop_duplicates(keep='first')
len(df)

Feature Engg

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# Initialize stemmer
ps = PorterStemmer()

In [ ]:
def transform_text(text):
    
    # Lowercase
    text = text.lower()
    
    # Tokenization
    text = nltk.word_tokenize(text)
    
    # Remove special characters and keep only alphanumeric
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    
    # Remove stopwords and punctuation
    text = y[:]
    y.clear()


    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    
    # Stemming
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    
    return " ".join(y)


In [ ]:
df['transform_text'] = df['text'].apply(transform_text)
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfid = TfidfVectorizer(max_features=500)

In [ ]:
X = tfid.fit_transform(df['transformed_text']).toarray()
y = df['target'].values

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=2)

Model Training

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
svc = SVC(kernel="sigmoid", gamma= 1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear',penalty='l1')
rfc = RandomForestClassifier(n_estimators=50,random_state=2)
abc = AdaBoostClassifier(n_estimators= 50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50, random_state=2)
xgb = XGBClassifier(n_estimators= 50, random_state = 2)



In [ ]:
clfs = {
    'SVC': svc,
    'KNN': knc,
    'NB' : mnb,
    'LR' : dtc,
    'RF' : lrc,
    'Adaboost': abc,
    'Bgc': bc,
    'ETC' : etc,
    'GBDT' : gbdt,
    'xgb' : xgb
}

Model Evaluation

from sklearn.metrics import accuracy_score, precision_score
def train_classifier(clfs, X_train, y_train, X_test, y_test):
    clfs.fit(X_train,y_train)
    y_pred = clfs.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    return accuracy, precision

In [ ]:
accuracy_score = []
precision_score = []
for name, clfs in clfs.items():
    current_accuracy, current_precision = train_classifier(clfs,X_train,y_train,X_test,y_test)
    print()
    print("For: ", name)
    print("Accuracy: ", current_accuracy)
    print("Precision: ", current_precision)

    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)